In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pyomo.environ import *

# 1. Load all Keras Models
SMR_fh2_model                = tf.keras.models.load_model('SMR_fh2.keras')
WE_ACM_H2_T_N_model          = tf.keras.models.load_model('WE_ACM_H2_T_N.keras')
fn2_fair_plpc_model          = tf.keras.models.load_model('fn2_fair_plpc.keras')
Fn2_P_Fair_model             = tf.keras.models.load_model('Fn2_P_Fair.keras')
Haber_f_model                = tf.keras.models.load_model('Haber_f.keras')
my_model7                    = tf.keras.models.load_model('my_model7.h5')
dist_column_f_ffeed_fh2o_model   = tf.keras.models.load_model('dist_column_f_ffeed_fh2o.keras')
dist_column_xnh3_ffeed_fh2o_model = tf.keras.models.load_model('dist_column_xnh3_ffeed_fh2o.keras')

SMR_energy_model    = tf.keras.models.load_model('SMR_ne_cal_s.keras')
WE_energy_model     = tf.keras.models.load_model('WE_ACM_NE_T_N.keras')
CAS_energy_model    = tf.keras.models.load_model('CASU_ne_W.keras')
MAS_energy_model    = tf.keras.models.load_model('MASU_ne_kW.keras')
HB_energy_model     = tf.keras.models.load_model('HB_ne_cal_s.keras')
Dist_energy_model   = tf.keras.models.load_model('dist_column_ne_ffeed_fh2o.keras')

CO_model   = tf.keras.models.load_model('CO_SMR.keras')
CO2_model  = tf.keras.models.load_model('CO2_SMR.keras')

# Load and scale SMR CO and CO2 data for the external functions
from sklearn.preprocessing import MinMaxScaler
file_path = 'CO_CO2_SMR.xlsx'
data = pd.read_excel(file_path)
data_array = data.to_numpy()
X_smr = data_array[:, [0, 1, 2, 3]]  # Input features
y_co = data_array[:, 4].reshape(-1, 1)  # Output feature (CO emissions)
y_co2 = data_array[:, 5].reshape(-1, 1)  # Output feature (CO2 emissions)

scaler_X_smr = MinMaxScaler().fit(X_smr)
scaler_y_co = MinMaxScaler().fit(y_co)
scaler_y_co2 = MinMaxScaler().fit(y_co2)

In [2]:
### Make a prediction to test the keras file
z = SMR_fh2_model.predict([[100, 300, 1100, 35]])
print (z)

1/1 [==============================] - 1s 1s/step
[[373.59094]]


In [3]:
# Define external functions for Keras model predictions
def smr_h2_pred(FNG, FH2O_SMR, Tsr, Psr):
    return SMR_fh2_model.predict(np.array([[FNG, FH2O_SMR, Tsr, Psr]]))[0,0]

def we_h2_pred(T_WE, N_WE):
    return WE_ACM_H2_T_N_model.predict(np.array([[T_WE, N_WE]]))[0,0]

def cas_n2_pred(fAir_CAS, Plpc_CAS):
    return fn2_fair_plpc_model.predict(np.array([[fAir_CAS, Plpc_CAS]]))[0,0] / 15.0

def mas_n2_pred(P_MAS, Fair_MAS):
    return Fn2_P_Fair_model.predict(np.array([[P_MAS, Fair_MAS]]))[0,0]

def haber_ammonia_pred(n2_flow, h2_flow):
     # Need to handle the case where n2_flow or h2_flow might be zero if a route is not selected
    if n2_flow == 0 or h2_flow == 0:
        return 0
    return Haber_f_model.predict(np.array([[n2_flow, h2_flow]]))[0,0]

def microwave_ammonia_pred(n2_flow, h2_flow):
    # Need to handle the case where n2_flow might be zero if a route is not selected
    if n2_flow == 0:
        return 0
    ratio = h2_flow / n2_flow
    inlet_gas_flow = (n2_flow * 0.0821 * 593.15 / (1.19 * 60 * 5.44)) + (h2_flow * 0.0821 * 593.15 / (1.19 * 60 * 5.44))
    ammonia_molar_fraction = my_model7.predict(np.array([[320, 80, ratio, inlet_gas_flow]]))[0,0]
    x = ammonia_molar_fraction * (n2_flow + h2_flow) / (2 + 2 * ammonia_molar_fraction)
    return 2 * x

def dist_total_flow_pred(ammonia_presep, FH2O_dist):
    return dist_column_f_ffeed_fh2o_model.predict(np.array([[ammonia_presep, FH2O_dist]]))[0,0]

def dist_ammonia_mf_pred(ammonia_presep, FH2O_dist):
    return dist_column_xnh3_ffeed_fh2o_model.predict(np.array([[ammonia_presep, FH2O_dist]]))[0,0]

def smr_energy_pred(FNG, FH2O_SMR, Tsr, Psr):
    return abs(SMR_energy_model.predict(np.array([[FNG, FH2O_SMR, Tsr, Psr]]))[0,0]) * 4.2

def we_energy_pred(T_WE, N_WE):
    return abs(WE_energy_model.predict(np.array([[T_WE, N_WE]]))[0,0]) * 10

def cas_energy_pred(fAir_CAS, Plpc_CAS):
    return abs(CAS_energy_model.predict(np.array([[fAir_CAS, Plpc_CAS]]))[0,0]) / 15.0

def mas_energy_pred(P_MAS, Fair_MAS):
    return abs(MAS_energy_model.predict(np.array([[P_MAS, Fair_MAS]]))[0,0]) * 1000

def hb_energy_pred(n2_flow, h2_flow):
     # Need to handle the case where n2_flow or h2_flow might be zero if a route is not selected
    if n2_flow == 0 or h2_flow == 0:
        return 0
    return abs(HB_energy_model.predict(np.array([[n2_flow, h2_flow]]))[0,0]) * 4.2

def dist_energy_pred(final_ammonia, FH2O_dist):
    return abs(Dist_energy_model.predict(np.array([[final_ammonia, FH2O_dist]]))[0,0]) * 4.2

def smr_co_pred(FNG, FH2O_SMR, Tsr, Psr):
    scaled_input = scaler_X_smr.transform(np.array([[FNG, FH2O_SMR, Tsr, Psr]]))
    scaled_pred = CO_model.predict(scaled_input)
    return scaler_y_co.inverse_transform(scaled_pred)[0,0]

def smr_co2_pred(FNG, FH2O_SMR, Tsr, Psr):
    scaled_input = scaler_X_smr.transform(np.array([[FNG, FH2O_SMR, Tsr, Psr]]))
    scaled_pred = CO2_model.predict(scaled_input)
    return scaler_y_co2.inverse_transform(scaled_pred)[0,0]

In [5]:
# Constants
ammonia_target = 100.0  # kmol/h
membrane_area = 100.0
pressure_diff = 0.3e6
ammonia_perm = 2.8e-8
h2_n2_perm_sum = 6.57e-9
MW_ENERGY_CONSTANT = 23 * 173 * 1000

# 2. Model Construction
model = ConcreteModel()

# Constants
ammonia_target = 100.0
M = 1e6  # Big-M value

# Binary variables with initial values
model.SMR = Var(domain=Binary, initialize=1)
model.WE = Var(domain=Binary, initialize=0)
model.CAS = Var(domain=Binary, initialize=1)
model.MAS = Var(domain=Binary, initialize=0)
model.HB = Var(domain=Binary, initialize=1)
model.MW = Var(domain=Binary, initialize=0)
model.Dist = Var(domain=Binary, initialize=1)
model.Mem = Var(domain=Binary, initialize=0)

In [6]:
# Continuous variables with relaxed bounds
model.FNG = Var(bounds=(80, 120), initialize=100)
model.FH2O_SMR = Var(bounds=(255, 375), initialize=300)
model.Tsr = Var(bounds=(900, 1100), initialize=1000)
model.Psr = Var(bounds=(29.6, 46.8), initialize=35)

model.T_WE = Var(bounds=(50, 80), initialize=65)
model.N_WE = Var(bounds=(10, 50), initialize=30)

model.fAir_CAS = Var(bounds=(12000, 16000), initialize=15000)
model.Plpc_CAS = Var(bounds=(0.01, 1.36), initialize=0.7)

model.P_MAS = Var(bounds=(5, 15), initialize=10)
model.Fair_MAS = Var(bounds=(2000, 2487.50), initialize=2000)

model.FH2O_dist = Var(bounds=(100, 200), initialize=150)

# Output variables
model.h2_flow_SMR = Var(bounds=(0, 2000))
model.h2_flow_WE = Var(bounds=(0, 2000))
model.n2_flow_CAS = Var(bounds=(0, 2000))
model.n2_flow_MAS = Var(bounds=(0, 2000))
model.ammonia_flow_HB = Var(bounds=(0, 2000))
model.ammonia_flow_MW = Var(bounds=(0, 2000))
model.dist_total_flow = Var(bounds=(0, 2000))
model.dist_ammonia_mf = Var(bounds=(0, 2000))

# ==============================================
# 3. Simplified Constraints
# ==============================================

# Route selection
model.hydrogen_route = Constraint(expr= model.SMR + model.WE == 1)
model.nitrogen_route = Constraint(expr= model.CAS + model.MAS == 1)
model.reactor_route = Constraint(expr= model.HB + model.MW == 1)
model.separation_route = Constraint(expr= model.Dist + model.Mem == 1)

# Process models
model.h2_flow_SMR_constr = Constraint(expr= model.h2_flow_SMR == smr_h2_poly(model.FNG, model.FH2O_SMR, model.Tsr, model.Psr))
model.h2_flow_WE_constr = Constraint(expr= model.h2_flow_WE == we_h2_poly(model.T_WE, model.N_WE))
model.h2_flow = Expression(expr= model.SMR * model.h2_flow_SMR + model.WE * model.h2_flow_WE)

model.n2_flow_CAS_constr = Constraint(expr= model.n2_flow_CAS == cas_n2_poly(model.fAir_CAS, model.Plpc_CAS))
model.n2_flow_MAS_constr = Constraint(expr= model.n2_flow_MAS == mas_n2_poly(model.P_MAS, model.Fair_MAS))
model.n2_flow = Expression(expr= model.CAS * model.n2_flow_CAS + model.MAS * model.n2_flow_MAS)

model.ammonia_flow_HB_constr = Constraint(expr= model.ammonia_flow_HB == haber_ammonia_poly(model.n2_flow, model.h2_flow))
model.ammonia_flow_MW_constr = Constraint(expr= model.ammonia_flow_MW == microwave_ammonia_poly(model.n2_flow, model.h2_flow))
model.ammonia_presep = Expression(expr= model.HB * model.ammonia_flow_HB + model.MW * model.ammonia_flow_MW)

model.final_ammonia = Expression(expr= 
    model.Dist * model.ammonia_presep +
    model.Mem * model.ammonia_presep * 0.8)


# Energy variables
model.smr_energy = Var(bounds=(0, None))
model.we_energy = Var(bounds=(0, None))
model.cas_energy = Var(bounds=(0, None))
model.mas_energy = Var(bounds=(0, None))
model.hb_energy = Var(bounds=(0, None))
model.dist_energy = Var(bounds=(0, None))

# Energy constraints
model.smr_energy_constr = Constraint(expr= model.smr_energy == smr_energy_poly(model.FNG, model.FH2O_SMR, model.Tsr, model.Psr))
model.we_energy_constr = Constraint(expr= model.we_energy == we_energy_poly(model.T_WE, model.N_WE))
model.cas_energy_constr = Constraint(expr= model.cas_energy == cas_energy_poly(model.fAir_CAS, model.Plpc_CAS))
model.mas_energy_constr = Constraint(expr= model.mas_energy == mas_energy_poly(model.P_MAS, model.Fair_MAS))
model.hb_energy_constr = Constraint(expr= model.hb_energy == hb_energy_poly(model.n2_flow, model.h2_flow))
model.dist_energy_constr = Constraint(expr= model.dist_energy == dist_energy_poly(model.ammonia_presep, model.FH2O_dist))

# Emissions variables
model.smr_co = Var(bounds=(0, None))
model.smr_co2 = Var(bounds=(0, None))

# Emissions constraints
model.smr_co_constr = Constraint(expr= model.smr_co == smr_co_poly(model.FNG, model.FH2O_SMR, model.Tsr, model.Psr))
model.smr_co2_constr = Constraint(expr= model.smr_co2 == smr_co2_poly(model.FNG, model.FH2O_SMR, model.Tsr, model.Psr))



# Production target
model.ammonia_target_constraint = Constraint(expr= model.final_ammonia >= 0.95 * ammonia_target)

In [7]:
# Define the EIP
def _EIP_total(m):
    # Energy terms
    e_SMR = m.SMR * m.smr_energy
    e_WE  = m.WE * m.we_energy
    e_CAS = m.CAS * m.cas_energy
    e_MAS = m.MAS * m.mas_energy
    e_HB  = m.HB * m.hb_energy
    e_MW  = m.MW * MW_ENERGY_CONSTANT
    e_D   = m.Dist * m.dist_energy
    e_Mem = m.Mem * 0 # Membrane energy is 0

    # Emission terms (SMR branch only relevant for CO/CO2)
    E_CH4    = m.SMR  * m.FNG * 16 * 265e-3
    E_H2O_SMR= m.SMR  * (m.FH2O_SMR + m.FH2O_dist) * 18 * 0.026e-3
    E_CO     = m.SMR  * m.smr_co  * 28 * 8.364e-3
    E_CO2    = m.SMR  * m.smr_co2 * 44 * 5.454e-3
    # H2O for WE calculated from h2 flow
    H2O_WE_flow = m.h2_flow_WE / 0.881759433
    #print (H2O_WE_flow)
    E_H2O_WE = m.WE   * H2O_WE_flow * 18 * 0.026e-3

    E_HB     = m.HB   * e_HB * 0.10e-6
    E_MW     = m.MW   * e_MW * 0.015e-6
    E_MAS    = m.MAS  * e_MAS * 0.015e-6
    E_WE_energy = m.WE * e_WE * 0.015e-6 # WE energy contribution
    E_EN     = (e_SMR + e_CAS + e_D + e_Mem) * 0.10e-6 # Other energy contributions
    E_minor  = 0.2 * (m.SMR + m.WE + m.CAS + m.MAS + m.HB + m.MW + m.Dist + m.Mem)


    return (E_CH4 + E_H2O_SMR + E_CO + E_CO2
            + E_H2O_WE + E_HB + E_MW + E_MAS
            + E_WE_energy + E_EN + E_minor)

model.EIP = Expression(rule=_EIP_total)
model.objective = Objective(expr=model.EIP, sense=minimize)

In [8]:
def solve_model():
    solver = SolverFactory('scip')
#    solver.options = {
#        'limits/time': 300,
#        'limits/gap': 0.01
#    }
    results = solver.solve(model, tee=True)
    
    if str(results.solver.termination_condition) == "optimal":
        print("\n=== Optimal Solution Found ===")
        print(f"Total EIP: {value(model.EIP):.2f}")
        print("\nSelected Routes:")
        print(f"Hydrogen: {'Steam Methane Reforming' if value(model.SMR) == 1 else 'Water Electrolysis'}")
        print(f"Nitrogen: {'Cryogenic Air Separation' if value(model.CAS) == 1 else 'Membrane Assisted Separation'}")
        print(f"Reactor: {'Haber-Bosch Reactor' if value(model.HB) == 1 else 'Microwave Reactor'}")
        print(f"Separation: {'Distillation Column Separation' if value(model.Dist) == 1 else 'Membrane Assisted Downstream Separation'}")
    else:
        from pyomo.util.infeasible import log_infeasible_constraints
        log_infeasible_constraints(model)
        print("Optimization failed - check infeasible.log")

solve_model()

SCIP version 9.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.1] [GitHash: bebb64304e]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 1cc71921]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  ZLIB 1.2.13          General purpose compression library by J. Gailly and M. Adler (zlib.net)
  GMP 6.3.0            GNU Multiple Precision Arithmetic Library developed by T. Granlund (gmplib.org)
  ZIMPL 3.6.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.2.1         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) (built with TBB) [GitHash: 3f1f0d53]
  bliss 0.77           Computing Graph Automor